# Loop Workflow Example 1 with flow calcs pasted at the end

> * High level approach to making a 3D model from just a bounding box you can draw
> * To run this notebook for the first time, there are some dependencies needed to use the interactive map. Execute the cell immediately below and restart jupyter.

In [ ]:
#if not already installed :
#!conda install -c conda-forge leafmap ipyleaflet ipywidgets -y 
# may need to restart jupyter server
import warnings
warnings.filterwarnings('ignore')

## Leaflet Maps

In [ ]:
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt

# load last saved map area and model engine (if they exist)
if(not os.path.isdir('../scratch/')):
    os.mkdir('../scratch/')
if(os.path.isfile('../scratch/last_choices.txt')):
    f=open('../scratch/last_choices.txt','r')
    contents =f.readlines()
    f.close()
    default_map=contents[0].replace("\n","")
    default_engine=contents[1].replace("\n","")
else:
    default_map='Turner_Syncline'
    default_engine='loopstructural'

options=['Draw Your Own','Last Area Drawn']

if(not default_map in options):
    default_map= options[0]

map_choice=widgets.Dropdown(
    options=options,
    value=default_map,
    description='Map area:',
    disabled=False,
)
display(map_choice)

In [ ]:
test_data_name=map_choice.value
print(test_data_name)

In [ ]:
import leafmap
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
dst_crs = "epsg:28350"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)

if(not test_data_name =='Draw Your Own'):
    if(test_data_name=='Last Area Drawn'):
        last_coords=pd.read_csv('../scratch/last_area.csv')
        display(last_coords)
        minx=last_coords.iloc[0]['minx']
        miny=last_coords.iloc[0]['miny']
        maxx=last_coords.iloc[0]['maxx']
        maxy=last_coords.iloc[0]['maxy']
    elif(not test_data_name =='Draw Your Own'):
        y_point_list = [miny, miny, maxy, maxy, maxy]
        x_point_list = [minx, maxx, maxx, minx, minx]
        bbox_geom = Polygon(zip(x_point_list, y_point_list))
        polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
        polygon_ll=polygon.to_crs(src_crs)

        minx=polygon_ll.total_bounds[0]
        maxx=polygon_ll.total_bounds[2]
        miny=polygon_ll.total_bounds[1]
        maxy=polygon_ll.total_bounds[3]

        minlong=minx
        maxlong=maxx
        minlat=miny
        maxlat=maxy
        #print("x",polygon_ll.total_bounds[0])
        st_bbox=[minlong,minlat,maxlong,maxlat]
        lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
        lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
        bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
        rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    

    bbox2=str(minx)+","+str(miny)+","+str(maxx)+","+str(maxy)
    y_point_list = [miny, miny, maxy, maxy, maxy]
    x_point_list = [minx, maxx, maxx, minx, minx]
    bbox_geom = Polygon(zip(x_point_list, y_point_list))
    polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
    polygon_ll=polygon.to_crs(src_crs)

    minlong=polygon_ll.total_bounds[0]
    maxlong=polygon_ll.total_bounds[2]
    minlat=polygon_ll.total_bounds[1]
    maxlat=polygon_ll.total_bounds[3]
    
    minlong=minx
    maxlong=maxx
    minlat=miny
    maxlat=maxy

    lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
    lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
    bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
    rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    center=(minlat+((maxlat-minlat)/2),minlong+((maxlong-minlong)/2))
else:
    center=(-22.6,117.3)

    
m =leafmap.Map( center=center, zoom=8,scroll_wheel_zoom=True)
m.add_basemap( basemap='OpenTopoMap')

m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpgeop15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Geology data from GSWA',
    name='geology',
    shown=True)
aurl='https://www.loopwms.xyz/geoserver/loop/wms?'
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpstrucl15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Linear features data from GSWA',
    name='faults/folds')

m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='waroxi_wa_4326_bed',
    format='image/png',
    transparent=True,
    attribution='Outcrop data from GSWA',
    name='outcrops')



if( not test_data_name =='Draw Your Own'):
    m.add_gdf(rect, layer_name="Last Rectangle", 
                   fill_colors=['orange'])
m

In [ ]:
polyclip=False # implemented in future version

if(test_data_name=='Draw Your Own' or test_data_name=='Last Area Drawn'):
    if(test_data_name=='Draw Your Own'):
        src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
        dst_crs = "epsg:28350" # coordinate system for example data
        
        rect=m.draw_features

        if(len(rect)==0):
            minlong=117.13698
            maxlong=117.564464
            minlat= -22.690712
            maxlat=-22.396454

        else:
            minlong=rect[0]['geometry']['coordinates'][0][0][0]
            maxlong=rect[0]['geometry']['coordinates'][0][2][0]
            minlat= rect[0]['geometry']['coordinates'][0][0][1]
            maxlat= rect[0]['geometry']['coordinates'][0][1][1]    
    else:
        use_roi_clip=False
        roi_clip_path=''

    bounds=(minlong,maxlong,minlat,maxlat)


    lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
    lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
    bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
    mbbox = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 
    bbox=mbbox.total_bounds
    st_bbox=[bbox[0],bbox[1],bbox[2],bbox[3]]
    print(src_crs,mbbox.total_bounds)
    mbbox=mbbox.to_crs(dst_crs)
    print(dst_crs,mbbox.total_bounds)
    
    f=open('../scratch/last_area.csv','w') 
    ostr='minx,miny,maxx,maxy\n'
    f.write(ostr)
    ostr=str(minlong)+','+str(minlat)+','+str(maxlong)+','+str(maxlat)+'\n'
    f.write(ostr)
    f.close()
    

## Map2Loop

In [ ]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from datetime import datetime
import time
import warnings


t0 = time.time()
warnings.filterwarnings('ignore')
nowtime=datetime.now().isoformat(timespec='minutes')   
model_name=nowtime.replace(":","-").replace("T","-")
proj = Project(
                loopdata_state="WA",
                overwrite="true",
                verbose_level=VerboseLevel.NONE,
                project_path=model_name,
                working_projection="EPSG:28350",
                )

proj.update_config(
                    out_dir=model_name,
                    bbox_3d={
                         "minx": mbbox.total_bounds[0], #500000,
                         "miny": mbbox.total_bounds[1], #7490000,
                         "maxx": mbbox.total_bounds[2], #545000,
                         "maxy": mbbox.total_bounds[3], #7520000,
                         "base": -4800,
                         "top": 1200,
                    },
                    #loopFilename='test.loop3d',
                    run_flags={                        
                        'aus': True,
                        'close_dip': -999,
                        'contact_decimate': 5,
                        'contact_dip': -999,
                        'contact_orientation_decimate': 5,
                        'deposits': "Fe,Cu,Au,NONE",
                        'dist_buffer': 10,
                        'dtb': '',
                        'fat_step': 750,
                        'fault_decimate': 5,
                        'fault_dip': -999,
                        'fold_decimate': 5,
                        'interpolation_scheme': 'scipy_rbf',
                        'interpolation_spacing': 500,
                        'intrusion_mode': 0,
                        'max_thickness_allowed': 10000,
                        'min_fault_length': 5000,
                        'misorientation': 30,
                        'null_scheme': 'null',
                        'orientation_decimate': 0,
                        'pluton_dip': 45,
                        'pluton_form': 'domes',
                        'thickness_buffer': 5000,
                        'use_fat': False,
                        'use_interpolations': False,
                        'fault_orientation_clusters':2,
                        'fault_length_clusters':2,
                        'map2graph':True,
                        'granular_map2graph':True
                    },
                  )
proj.config.c_l['intrusive']='intrusive'
#proj.config.c_l['intrusive']='mafic intrusive'
proj.workflow['fold_axial_traces']=False
proj.run()
t1 = time.time()
print("m2l",(t1-t0)/60.0,"minutes")

## Loop Structural

In [ ]:
# Define project pathing from m2l

proj_path = proj.config.project_path
graph_path = proj.config.graph_path
tmp_path = proj.config.tmp_path
dtm_path=model_name+'/dtm/'
vtk_path=model_name+'/vtk/'
output_path = proj.config.output_path

# Define project bounds
minx,miny,maxx,maxy = proj.config.bbox
model_base = proj.config.bbox_3d['base']
model_top = proj.config.bbox_3d['top']


In [ ]:
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer
from datetime import datetime
import os
import time
from datetime import datetime
import shutil
import logging
#logging.getLogger().setLevel(logging.ERROR)
import lavavu
import numpy as np
import math
from map2loop.m2l_utils import save_dtm_mesh, save_dtm_ascii
from map2loop.map import MapUtil
import rasterio
from scipy.interpolate import RegularGridInterpolator

surface_verts = {}

def function(xyz,tri,name): # for saving out vtk files
    xyz = np.copy(xyz)
    tri = np.copy(tri)
    nanmask = np.all(~np.isnan(xyz),axis=1)
    vert_idx = np.zeros(xyz.shape[0],dtype=int) -1
    vert_idx[nanmask] = np.arange(np.sum(nanmask))
    tri[:] = vert_idx[tri]
    tri = tri[np.all(tri > -1,axis=1),:]
    xyz = xyz[nanmask,:]
    surface_verts[name] = (xyz,tri)
    
def mask(xyz): # for clipping strat to surface dtm
    from map2loop.map import MapUtil
    import rasterio
    import os
    dtm_map = MapUtil(proj.config.bbox_3d,dtm=rasterio.open(os.path.join(dtm_path,'dtm_rp.tif')))
    xyz=model.rescale(xyz,inplace=False)
    dtmv = dtm_map.evaluate_dtm_at_points((xyz[:,:2]))
    return xyz[:,2]<dtmv



t1a = time.time()

filename=vtk_path+'/'+'surface_name_{}.vtk'

save_dtm_mesh(dtm_path,vtk_path)

save_dtm_ascii(dtm_path)
               
f=open(tmp_path+'/bbox.csv','w')
f.write('minx,miny,maxx,maxy,lower,upper\n')
ostr='{},{},{},{},{},{}\n'.format(minx,miny,maxx,maxy,model_base,model_top)
f.write(ostr)
f.close()


fault_params = {'interpolatortype':'FDI',
                 'nelements':1e5,
                }
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                    'nelements':1e5,  # how many tetras/voxels
                    }
model, m2l_data = GeologicalModel.from_map2loop_directory(proj_path,
                                                          fault_params=fault_params,
                                                          rescale=False,
                                                          foliation_params=foliation_params)
model.update()
                                       
#model.to_file(output_path + "/model.pickle")    


clip_on_dtm=True
if(clip_on_dtm):
    dtm = rasterio.open(dtm_path+'/dtm_rp.tif')
    dtm_val = dtm.read(1)
    x=np.linspace(dtm.bounds[0],dtm.bounds[2],dtm_val.shape[1])
    y=np.linspace(dtm.bounds[1],dtm.bounds[3],dtm_val.shape[0])
    dtm_interpolator = RegularGridInterpolator((x,y),np.rot90(dtm_val,3))
    model.dtm = lambda xyz : dtm_interpolator(xyz[:,:2])

view = LavaVuModelViewer(model,vertical_exaggeration=1) 
view.nsteps = np.array([200,200,200])
view.nsteps=np.array([50,50,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(function=function,filename=filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(function=function,filename=filename,strati=False,displacement_cmap = 'rainbow')
view.add_dtm()
view.lv.webgl(vtk_path+model_name)
view.nsteps = np.array([200,200,200])
view.add_model()

view.lv.control.Range('alpha', label="Global Opacity")
view.lv.control.DualRange(['xmin', 'xmax'], label="x clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['ymin', 'ymax'], label="y clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['zmin', 'zmax'], label="z clip", step=0.01, values=[0.0,1.0])
view.lv.control.Range(command='background', range=(0,1), step=0.1, value=0.8)
view.lv.control.show() #Show the control panel, including the viewer window

for layer in surface_verts:
    f=open(vtk_path+layer.replace("_iso_0.000000","")+'.obj','w')
    vert=surface_verts[layer][0].shape[0]
    tri=surface_verts[layer][1].shape[0]
    #print(layer.replace("_iso_0.000000",""),vert,tri)
    for v in range(0,vert):
        ostr = "v {} {} {}\n"\
            .format(surface_verts[layer][0][v][0],surface_verts[layer][0][v][1],surface_verts[layer][0][v][2])
        f.write(ostr)
    for t in range(0,tri):
        ostr = "f {} {} {}\n"\
                .format(surface_verts[layer][1][t][0]+1,surface_verts[layer][1][t][1]+1,surface_verts[layer][1][t][2]+1)
        f.write(ostr)
    first=False
    f.close()
    
view.interactive()  

t2 = time.time()
print("m2l",(t1-t0)/60.0,"LoopStructural",(t2-t1a)/60.0,"Total",(t2-t1a+t1-t0)/60.0,"minutes")

In [ ]:
model.feature_name_index

In [ ]:
save_vtk=False

if(save_vtk):
    from LoopStructural.visualisation.vtk_exporter import VtkExporter
    view = VtkExporter(model,vtk_path)

    view.nsteps = np.array([200,200,200])

    view.nelements = 1e5
    view.add_model_surfaces(function=function,filename=filename,faults=False)
    view.nelements=1e6
    view.add_model_surfaces(function=function,filename=filename,strati=False,displacement_cmap = 'rainbow')

## Save voxel model

In [ ]:
save_voxels=False
if(save_voxels  ):  
    voxel_size=500
    sizex=int((maxx-minx)/voxel_size)
    sizey=int((maxy-miny)/voxel_size)
    sizez=int((model_top-model_base)/voxel_size)
    print('voxel_size=',voxel_size,', saved in Z,Y,X order 16 bit unisgned, X(height)=',sizex,', Y(#ims)=',sizey,', Z(width)=',sizez)
    print('lower south west corner: west=',minx,', south=',miny,', lower=',model_base)
    voxels=model.evaluate_model(model.regular_grid(nsteps=(sizey,sizex,sizez),shuffle=False),scale=False)
    voxels.astype('int16').tofile(model_name+'/vtk/voxels.raw')
    print('voxels saved as',model_name+'/vtk/voxels.raw')

## Save Geoscience Analyst Model

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single geoh5 file that GeoscienceAnalyst can read
#Requires installation of https://github.com/MiraGeoscience/geoh5py 
#(note .geoh5 meshes are 0 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from geoh5py.objects import BlockModel
from geoh5py.workspace import Workspace
from geoh5py.objects import Surface

def hextofloats(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return tuple(int(h[i:i + 2], 16) / 255. for i in (1, 3, 5))  # skip '#'

def geoh5_create_surface_data(obj_path_dir,colour_path):

    h5file_path = obj_path_dir+"/loop.geoh5"

    workspace = Workspace(h5file_path)
    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    colour_index=0
    all_sorts = pd.read_csv(os.path.join(colour_path, 'all_sorts_clean.csv'), ",")
    all_sorts=all_sorts.set_index('code')
    colour_map=open(obj_path_dir+'/loop_colour_map.clr','w')
    colour_map.write('{\tStart\tRed\tGreen\tBlue\t}\n')

    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
            if(len(i)>0 and len(v)>0):
                # Create a geoh5 surface
                surface = Surface.create(
                          workspace, name=file.replace('.obj',''), vertices=v, cells=i
                            )
                if('Fault_' in file or 'dtm' in file):
                    colours=np.ones(surface.n_cells)*99
                else:
                    colours=np.ones(surface.n_cells)*colour_index
                    rgb=hextofloats(all_sorts.loc[file.replace('.obj','')]['colour'])
                    colour_map.write('{}\t{}\t{}\t{}\n'.format(colour_index,rgb[0],rgb[1],rgb[2]))
                    colour_index=colour_index+1
                    

                surface.add_data({
                    "colour_index": {
                        "association":"CELL",
                        "values": colours
                    }
                })
                
                workspace.save_entity(surface)
    workspace.close()
                
    colour_map.write('{}\t{}\t{}\t{}\n'.format(99,1,1,1))
    colour_map.close()
    print("colour map saved as:",obj_path_dir+'/loop_colour_map.clr')


save_GA=True

if(save_GA):
    geoh5_create_surface_data(vtk_path,tmp_path)


In [ ]:
from LoopFlow import import_triangulation as it
from LoopFlow import calculate_flow as cf

bbox=it.create_bbox(minx,miny,maxx,maxy,model_base,model_top)
destination=model_name+'/'

In [ ]:
voxel_size=750
faults_only=False
                    
scenario={     #free-form scenario   
        'fault_node':1000.0,
        'geological_formation_slow':1000.0,
        'geological_formation_fast':1.0,
        'interformation_node':1000.0,

        'fault_formation':1000.0,
        'same_fault':1000.0,
        'fault_fault':1000.0,
        'interform_fault':1000.0,
        'interform_formation':100.0,
        'interform_interform':100.0,
        'same_interform':100.0,

        'fast_formation_code':['0']
        }
scenario='fast_faults' #pre-defined scenario
                        # fast_faults    
                        # fast_strat_contacts 
                        # fast_both  
                        # fault_barriers_not_paths  
                        # fault_barriers_but_paths_and_fast_strat
                        #homogeneous

source='west' # 'west','north', 'south', top', 'base', 'deep_line', 'point'
target='east' # 'west','north', 'south', top', 'base', 'deep_line', 'point'
fast_formation_code=['0'] #list of codes that should be 'fast'

sourcen=-1
targetn=103232

ptx=(bbox.loc[0].minx+bbox.loc[0].maxx)/2
pty=(bbox.loc[0].miny+bbox.loc[0].maxy)/2
ptz=bbox.loc[0].lower

f=open(destination+'/parameters.txt','w')
f.write('faults_only = {}\nscenario = {}\nsource = {}\nfast_litho = {}\npoint = {},{},{}\n'.format(faults_only,str(scenario),source,fast_formation_code,ptx,pty,ptz))
f.close()

Graw,df_nodes_raw,df_edges_raw=cf.graph_from_model(model,voxel_size,bbox,destination)

G,scenario=cf.assign_weights(Graw,scenario,source,target,fast_formation_code,faults_only,bbox,ptx,pty,ptz,voxel_size/2)
voxet_df,distance,path=cf.calculate_dist(G,df_nodes_raw,voxel_size,bbox,scenario,destination)
cf.calculate_paths(path,df_nodes_raw,scenario,destination)

In [ ]:
scenery=cf.calculate_scenery(G,model,df_nodes_raw,path,scenario,destination)    
cf.merge_outputs(voxet_df,df_nodes_raw,scenery,scenario,destination)

In [ ]:
sourcen=-1
targetn=-2
cf.calc_boykov_kolmogorov(G,sourcen,targetn,df_nodes_raw,df_edges_raw,scenario,destination)